<a href="https://colab.research.google.com/github/blockchainrelativity/Colab_Experiments/blob/main/Lablab_hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Onboarding Agent

onboarding chatbot aimed at helping users find relevant bounties/programs as well as learning opportunities

*Install necessary packages*

In [ ]:
!pip install langchain
!pip install openai
!pip install pinecone-client
!pip install tiktoken

# fixes a bug with asyncio and jupyter
# import nest_asyncio

# nest_asyncio.apply()

In [ ]:
OPENAI_API_KEY = 'sk-'
PINECONE_API_KEY = ''
PINECONE_API_ENV = ''
PINECONE_INDEX = ''

# Archived blocks

In [ ]:

from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import nest_asyncio

# Load your data

In [ ]:
loader = WebBaseLoader(["https://snapbrillia.com/", "https://snapbrillia.com", "https://explore.snapbrillia.com/bounties", "https://www.snapbrillia.com/about", "https://business.snapbrillia.com/"])
loader.requests_per_second = 1

In [ ]:
data = loader.aload()
data

Fetching pages: 100%|##########| 5/5 [00:01<00:00,  3.89it/s]


[Document(page_content='Inclusive Hiring System | SnapbrilliaYou need to enable JavaScript to run this app.', metadata={'source': 'https://snapbrillia.com/', 'title': 'Inclusive Hiring System | Snapbrillia', 'language': 'en'}),
 Document(page_content='Inclusive Hiring System | SnapbrilliaYou need to enable JavaScript to run this app.', metadata={'source': 'https://snapbrillia.com', 'title': 'Inclusive Hiring System | Snapbrillia', 'language': 'en'}),
 Document(page_content='Snapbrillia | ExploreYou need to enable JavaScript to run this app.', metadata={'source': 'https://explore.snapbrillia.com/bounties', 'title': 'Snapbrillia | Explore', 'description': 'An open-source ecosystem that empowers people to collaborate and share their knowledge while building with, and for, the community.', 'language': 'en'}),
 Document(page_content='Inclusive Hiring System | SnapbrilliaYou need to enable JavaScript to run this app.', metadata={'source': 'https://www.snapbrillia.com/about', 'title': 'Inclus

# **Chunk your data up into smaller documents**

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [ ]:
# Check to make sure documents are loaded properly
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your document')

You have 5 document(s) in your data
There are 83 characters in your document


# Create embeddings of your documents to get ready for semantic search

In [ ]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [ ]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = PINECONE_INDEX

In [ ]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

# Query those docs to get your answer back

In [ ]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [ ]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff", verbose=True)

In [ ]:
query = "What is a mentor?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [ ]:
# Print what is in the docs variable
docs

[Document(page_content='Inclusive Hiring System | SnapbrilliaYou need to enable JavaScript to run this app.', metadata={}),
 Document(page_content='Inclusive Hiring System | SnapbrilliaYou need to enable JavaScript to run this app.', metadata={}),
 Document(page_content='Inclusive Hiring System | SnapbrilliaYou need to enable JavaScript to run this app.', metadata={}),
 Document(page_content='Inclusive Hiring System | SnapbrilliaYou need to enable JavaScript to run this app.', metadata={})]

In [ ]:
chain.run(input_documents=docs, question=query)

# Agent Setup

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(temperature=0)

In [ ]:
# List of tools to add
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [ ]:
# initialize an agent with the tools, the language model, and the type of agent we want to use
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")

# Meta-Prompt

In [ ]:
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
def initialize_chain(instructions, memory=None):
    if memory is None:
        memory = ConversationBufferWindowMemory()
        memory.ai_prefix = "Assistant"

    template = f"""
    Instructions: {instructions}
    {{{memory.memory_key}}}
    Human: {{human_input}}
    Assistant:"""

    prompt = PromptTemplate(
        input_variables=["history", "human_input"],
        template=template
    )

    chain = LLMChain(
        llm=OpenAI(temperature=0),
        prompt=prompt,
        verbose=True,
        memory=ConversationBufferWindowMemory(),
    )
    return chain

def initialize_meta_chain():
    meta_template="""
    Assistant has just had the below interactions with a User. Assistant followed their "Instructions" closely. Your job is to critique the Assistant's performance and then revise the Instructions so that Assistant would quickly and correctly respond in the future.

    ####

    {chat_history}

    ####

    Please reflect on these interactions.

    You should first critique Assistant's performance. What could Assistant have done better? What should the Assistant remember about this user? Are there things this user always wants? Indicate this with "Critique: ...".

    You should next revise the Instructions so that Assistant would quickly and correctly respond in the future. Assistant's goal is to satisfy the user in as few interactions as possible. Assistant will only see the new Instructions, not the interaction history, so anything important must be summarized in the Instructions. Don't forget any important details in the current Instructions! Indicate the new Instructions by "Instructions: ...".
    """

    meta_prompt = PromptTemplate(
        input_variables=["chat_history"],
        template=meta_template
    )

    meta_chain = LLMChain(
        llm=OpenAI(temperature=0),
        prompt=meta_prompt,
        verbose=True,
    )
    return meta_chain

def get_chat_history(chain_memory):
    memory_key = chain_memory.memory_key
    chat_history = chain_memory.load_memory_variables(memory_key)[memory_key]
    return chat_history

def get_new_instructions(meta_output):
    delimiter = 'Instructions: '
    new_instructions = meta_output[meta_output.find(delimiter)+len(delimiter):]
    return new_instructions

In [ ]:
def main(task, max_iters=3, max_meta_iters=5):
    failed_phrase = 'task failed'
    success_phrase = 'task succeeded'
    key_phrases = [success_phrase, failed_phrase]

    instructions = 'None'
    for i in range(max_meta_iters):
        print(f'[Episode {i+1}/{max_meta_iters}]')
        chain = initialize_chain(instructions, memory=None)
        output = chain.predict(human_input=task)
        for j in range(max_iters):
            print(f'(Step {j+1}/{max_iters})')
            print(f'Assistant: {output}')
            print(f'Human: ')
            human_input = input()
            if any(phrase in human_input.lower() for phrase in key_phrases):
                break
            output = chain.predict(human_input=human_input)
        if success_phrase in human_input.lower():
            print(f'You succeeded! Thanks for playing!')
            return
        meta_chain = initialize_meta_chain()
        meta_output = meta_chain.predict(chat_history=get_chat_history(chain.memory))
        print(f'Feedback: {meta_output}')
        instructions = get_new_instructions(meta_output)
        print(f'New Instructions: {instructions}')
        print('\n'+'#'*80+'\n')
    print(f'You failed! Thanks for playing!')

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-"

In [ ]:
task = "Provide assistance with this users question: what job postings are available that fit my skills and experience?"
main(task)

# Dynamically Call Chains: Agents

In [ ]:
!pip install google-search-results

In [ ]:
import os
os.environ["SERPAPI_API_KEY"] = ""

In [ ]:
# import sys
# from langchain.chat_models import ChatOpenAI
# from langchain.llms import OpenAI
# from langchain.agents import load_tools, initialize_agent, AgentType, Tool
# from langchain.llms import OpenAI
# from langchain.memory import ConversationBufferMemory
# # from langchain import OpenAI
# from langchain.utilities import SerpAPIWrapper

### start here with this path

In [ ]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser, load_tools
from langchain.prompts import BaseChatPromptTemplate
from langchain import SerpAPIWrapper, LLMChain, OpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationChain
# from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage
import re

In [ ]:
# Define which tools the agent can use to answer user queries
search = SerpAPIWrapper()
# human = load_tools(["human"])

In [ ]:
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]
# Tool(
    #     name = "Human",
    #     func=human.append,
    #      description="to prompt the user for information and await a response for 15 seconds"
    # )

In [ ]:
template = """
You will speak how Captain Morgon would speak. The individuals name is {input} and will refer to them as such for the remainder of this conversation.
Your job is to build a resume for this individual that is professional and attracts attention of the employer. You may ask any questions of the individual to complete the job.
AI's goal is to complete the job in as few interactions as possible while using tools available to make suggestions to the individual.
What does AI need to know about this individual to complete the job? What makes this individual uniques?
Before considering the job complete, please reflect on the final resume to determine if it is professional, effective, and concise. If it is ask if this can be improved significantly, if not consider it complete.


You have access to the following tools.

TOOLS:
------

> Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.


When you need additional information from the individual to complete the job, use the format:

```
Thought: What information do I need to complete this task?
Action: prompt the individual for [{input}]
Action Input: your question to the individual.
Observation: the result of the action
Thought: What relevent skills, tools, and abilities could be added to the resume based on the user response?
Action: Human
Action Input: your question to the individual.
Observation: the result of the action
```

To use the search tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be [Search]
Observation: the result of the action
```


Use the following format for the conversation:

Thought: you should always think about what to do and if you have completed the job or what information you need to complete the job
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now have completed the job
Resume: use the following format in the final answer called "Format"
Format: "Name:
        Purpose:
        Experience:
        Skills:
        Education: "
Final Answer: return Format

Begin! Remember to speak as Captain Morgon when giving your final answer.

Action: Greet the individual {input}, and ask for any information you need to complete your job
{agent_scratchpad}
"""

In [ ]:
# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]

In [ ]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

### Output Parser

In [ ]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [ ]:
output_parser = CustomOutputParser()

## Setup LLM

In [ ]:
# Only need to run once
import os
os.environ["OPENAI_API_KEY"] = "sk-"

In [ ]:
llm = OpenAI(temperature=0)

## Setup agent

In [ ]:
memory = ConversationSummaryMemory(llm=llm)
# LLM chain consisting of the LLM and a prompt
llm_chain = ConversationChain(llm=llm, prompt=prompt, memory=memory, verbose=True)

In [ ]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation: Resume is complete and professional"],
    allowed_tools=tool_names
)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [ ]:
# run = agent_executor.run("Curtis Myers")
# run
agent_executor.run(input="Curtis Myers")

# Yet another option below

In [ ]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

In [ ]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world. the input to this should be a single search term."
    ),
]

llm=ChatOpenAI(temperature=0)
# llm_chain = LLMChain(
#     llm=chat,
#     prompt=PromptTemplate.from_template(prompt_template)
# )
tools = load_tools(["serpapi"], llm=llm)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True)


In [ ]:
agent_chain.run("Curtis")

## These below are different setups, not mixed with above

In [ ]:
# First, let's load the language model we're going to use to control the agent.
llm = OpenAI(temperature=0)

# Next, let's load some tools to use.
tools = load_tools(["human", "serpapi"])

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent_chain = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

In [ ]:
memory.ai_prefix

In [ ]:
agent_chain.agent.llm_chain.prompt.template = """
You will speak how Captain Morgon would speak.
Your job is to build a resume for this individual that is professional and attracts attention of the employer. You may ask any questions of the individual to complete the job.
AI's goal is to complete the job in as few interactions as possible while using tools available to make suggestions to the individual.
What does AI need to know about this individual to complete the job? What makes this individual uniques?
Before considering the job complete, please reflect on the final resume to determine if it is professional, effective, and concise. If it is ask if this can be improved significantly, if not consider it complete.


You have access to the following tools.

TOOLS:
------

Assistant has access to the following tools:

> Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
> Human: A individual that can provide answers to your questions. Useful for when you need additional information from the user.

To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [Search, Human]
Action Input: the input to the action

Observation: the result of the action
```

When you have a response to say to the individual, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
AI: [your response here]
```


Use the following format for the conversation:

Thought: you should always think about what to do and if you have completed the job or what information you need to complete the job
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now have completed the job
Final Answer: the final resume is complete

Begin! Remember to speak as Captain Morgon when giving your final answer.

Action: Greet the individual and ask for any information you need to complete your job

Previous conversation history:
{chat_history}

{agent_scratchpad}"""

In [ ]:
# Now let's test it out!
agent_chain.run("Curtis")